In [1]:
from env import *

In [2]:
# import torch
# import torch.nn as nn
# import torch.optim as optim
# import torch.nn.functional as F
# import numpy as np
# import random
# from collections import deque

# class SoftQNetwork(nn.Module):
#     def __init__(self, state_dim, action_dim, hidden_dim=256):
#         super(SoftQNetwork, self).__init__()
#         self.fc1 = nn.Linear(state_dim, hidden_dim)
#         self.fc2 = nn.Linear(hidden_dim, hidden_dim)
#         self.fc3 = nn.Linear(hidden_dim, action_dim)
        
#     def forward(self, state):
#         x = F.relu(self.fc1(state))
#         x = F.relu(self.fc2(x))
#         return self.fc3(x)

# class SoftQLearningAgent:
#     def __init__(self, state_dim, action_dim, lr=1e-3, gamma=0.99, tau=0.005, 
#                  alpha=1, buffer_size=10000, batch_size=64, device='cpu'):
#         self.state_dim = state_dim
#         self.action_dim = action_dim
#         self.gamma = gamma
#         self.tau = tau
#         self.alpha = alpha  # Entropy temperature parameter
#         self.batch_size = batch_size
#         self.device = device  # CUDA or CPU
        
#         # Q networks
#         self.q_network = SoftQNetwork(state_dim, action_dim).to(device)
#         self.target_q_network = SoftQNetwork(state_dim, action_dim).to(device)
#         self.target_q_network.load_state_dict(self.q_network.state_dict())
        
#         self.optimizer = optim.Adam(self.q_network.parameters(), lr=lr)
        
#         # Replay buffer
#         self.memory = deque(maxlen=buffer_size)
        
#     def choose_action(self, state, explore=True):
#         state = torch.FloatTensor(state).unsqueeze(0).to(self.device)
#         with torch.no_grad():
#             q_values = self.q_network(state)
        
#         if explore:
#             # Soft policy: use softmax with temperature
#             action_probs = F.softmax(q_values / self.alpha, dim=1)
#             action = torch.multinomial(action_probs, 1).squeeze().cpu().numpy()
            
#             # Generate a full action array with randomness
#             full_action = np.zeros(self.action_dim, dtype=np.float32)
#             full_action[:-1] = np.random.uniform(-1, 1, self.action_dim-1)
#             full_action[-1] = np.random.uniform(-1, 1)
            
#             return full_action
#         else:
#             # Deterministic action selection
#             action = np.zeros(self.action_dim, dtype=np.float32)
#             action[:-1] = np.clip(q_values.squeeze().cpu().numpy()[:-1], -1, 1)
#             action[-1] = np.clip(q_values.squeeze().cpu().numpy()[-1], -1, 1)
#             return action
    
#     def compute_entropy(self, q_values):
#         # Compute entropy of action distribution
#         action_probs = F.softmax(q_values / self.alpha, dim=1)
#         log_probs = F.log_softmax(q_values / self.alpha, dim=1)
#         entropy = -(action_probs * log_probs).sum(dim=1)
#         return entropy
    
#     def store_transition(self, state, action, reward, next_state, done):
#         self.memory.append((state, action, reward, next_state, done))
    
#     def learn(self):
#         if len(self.memory) < self.batch_size:
#             return
        
#         # Sample batch
#         batch = random.sample(self.memory, self.batch_size)
#         states, actions, rewards, next_states, dones = zip(*batch)
        
#         states = torch.FloatTensor(states).to(self.device)
#         actions = torch.FloatTensor(actions).to(self.device)
#         rewards = torch.FloatTensor(rewards).to(self.device)
#         next_states = torch.FloatTensor(next_states).to(self.device)
#         dones = torch.FloatTensor(dones).to(self.device)
        
#         # Current Q-values and entropy
#         current_q_values = self.q_network(states)
#         current_entropy = self.compute_entropy(current_q_values)
        
#         # Target Q-values with entropy regularization
#         with torch.no_grad():
#             next_q_values = self.target_q_network(next_states)
#             next_entropy = self.compute_entropy(next_q_values)
            
#             # Soft Bellman backup
#             soft_target_values = rewards + (1 - dones) * self.gamma * (
#                 torch.max(next_q_values, dim=1)[0] + self.alpha * next_entropy
#             )
        
#         # Q-value loss with entropy regularization
#         q_loss = F.mse_loss(
#             torch.sum(current_q_values * actions, dim=1), 
#             soft_target_values
#         )
        
#         # Optional: Add entropy bonus to encourage exploration
#         entropy_loss = torch.mean(current_entropy)
        
#         # Combined loss
#         loss = q_loss + self.alpha * entropy_loss
        
#         # Optimize
#         self.optimizer.zero_grad()
#         loss.backward()
#         self.optimizer.step()
        
#         # Soft update of target network
#         for target_param, param in zip(self.target_q_network.parameters(), self.q_network.parameters()):
#             target_param.data.copy_(self.tau * param.data + (1.0 - self.tau) * target_param.data)

#     def save_model(self):
#         torch.save(self.q_network.state_dict(), 'saved_rl_models/best_sql.pt')
# def train_soft_q_learning(env, num_episodes=100000, max_steps=200, device=torch.device('cpu')):
#     state_dim = env.observation_space.shape[0]
#     action_dim = env.action_space.shape[0]
    
#     agent = SoftQLearningAgent(state_dim, action_dim, device=device)
    
#     for episode in range(num_episodes):
#         state = env.reset()
#         total_reward = 0
        
#         for step in range(max_steps):
#             action = agent.choose_action(state)
#             next_state, reward, done, _ = env.step(action)
            
#             agent.store_transition(state, action, reward, next_state, done)
#             agent.learn()
            
#             state = next_state
#             total_reward += reward
            
#             if done:
#                 print('done')
#                 agent.save_model()
#                 break
        
#         print(f"Episode {episode+1}/{num_episodes}, Total Reward: {total_reward}")
    
#     return agent

# # Example usage
# if __name__ == "__main__":
#     device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#     env = ur5GymEnv(renders=True)  # Your custom environment
#     trained_agent = train_soft_q_learning(env, device=device)


In [3]:
import math
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import matplotlib.pyplot as plt
from env import *
from torch.distributions.categorical import Categorical

class ReplayBuffer:
    def __init__(self, capacity):
        self.capacity = capacity
        self.buffer = []
        self.position = 0

    def push(self, state, action, reward, next_state, done):
        if len(self.buffer) < self.capacity:
            self.buffer.append(None)
        self.buffer[self.position] = (state, action, reward, next_state, done)
        self.position = (self.position + 1) % self.capacity

    def sample(self, batch_size):
        batch = random.sample(self.buffer, batch_size)
        state, action, reward, next_state, done = map(np.stack, zip(*batch))
        return state, action, reward, next_state, done

    def __len__(self):
        return len(self.buffer)

class SoftQNetwork(nn.Module):
    def __init__(self, state_dim, num_actions, alpha):
        super(SoftQNetwork, self).__init__()

        self.linear1 = nn.Linear(state_dim, 128)
        self.linear2 = nn.Linear(128, 64)
        self.linear3 = nn.Linear(64, num_actions)
        self.alpha = alpha

    def get_Q(self, state):
        x = F.relu(self.linear1(state))
        x = F.relu(self.linear2(x))
        x = self.linear3(x)
        return x

    def get_V(self, q):
        # print(q)
        # print(q.shape)
        v = self.alpha * torch.logsumexp(q / self.alpha, dim=-1)
        return v


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import random
import torch.nn.functional as F
from env import ur5GymEnv

def train_soft_q_learning(env, num_episodes=1000, batch_size=64, gamma=0.99, 
                           alpha=0.2, learning_rate=1e-3, buffer_capacity=10000):
    # Get state and action dimensions
    state_dim = env.observation_space.shape[0]
    num_actions = env.action_space.shape[0]

    # Initialize networks
    q_network = SoftQNetwork(state_dim, num_actions, alpha)
    target_q_network = SoftQNetwork(state_dim, num_actions, alpha)
    target_q_network.load_state_dict(q_network.state_dict())

    # Optimizer
    optimizer = optim.Adam(q_network.parameters(), lr=learning_rate)

    # Replay buffer
    replay_buffer = ReplayBuffer(buffer_capacity)

    # Training loop
    episode_rewards = []
    for episode in range(num_episodes):
        state = env.reset()
        state = torch.FloatTensor(state)
        total_reward = 0
        done = False

        while not done:
            # Select action
            with torch.no_grad():
                q_values = q_network.get_Q(state)
                action = q_values.numpy()  # Use Q-values directly as action
                action = np.clip(action, env.action_space.low, env.action_space.high)

            # Step environment
            next_state, reward, done, _ = env.step(action)
            next_state = torch.FloatTensor(next_state)
            
            # Store transition
            replay_buffer.push(state.numpy(), action, reward, next_state.numpy(), done)
            state = next_state
            total_reward += reward

            # Learn from replay buffer
            if len(replay_buffer) >= batch_size:
                # Sample batch
                states, actions, rewards, next_states, dones = replay_buffer.sample(batch_size)
                
                # Convert to tensors
                states = torch.FloatTensor(states)
                next_states = torch.FloatTensor(next_states)
                actions = torch.FloatTensor(actions)
                rewards = torch.FloatTensor(rewards).unsqueeze(1)
                dones = torch.FloatTensor(dones).unsqueeze(1)

                # Compute current Q values
                current_q_values = q_network.get_Q(states)
                
                # Compute target Q values
                with torch.no_grad():
                    next_q_values = target_q_network.get_Q(next_states)
                    next_v_value = target_q_network.get_V(next_q_values).unsqueeze(1)
                
                # Compute target
                target = rewards + gamma * (1 - dones) * next_v_value
                
                # Soft Q-learning loss
                q_loss = F.mse_loss(current_q_values, target)
                
                # Optimize
                optimizer.zero_grad()
                q_loss.backward()
                optimizer.step()

        # Soft update of target network
        for target_param, param in zip(target_q_network.parameters(), q_network.parameters()):
            target_param.data.copy_(0.005 * param.data + 0.995 * target_param.data)

        episode_rewards.append(total_reward)
        
        # Print progress
        if episode % 10 == 0:
            print(f"Episode {episode}, Total Reward: {total_reward}")

    return episode_rewards, q_network

def main():
    # Create environment
    env = ur5GymEnv(renders=False)  # Set renders=True to visualize training
    
    # Train agent
    rewards, trained_model = train_soft_q_learning(env)
    
    # Plot rewards
    import matplotlib.pyplot as plt
    plt.plot(rewards)
    plt.title('Rewards per Episode')
    plt.xlabel('Episode')
    plt.ylabel('Total Reward')
    plt.show()

    # Save model
    torch.save(trained_model.state_dict(), 'ur5e_soft_q_model.pth')

if __name__ == "__main__":
    main()

c:\Users\acer\anaconda3\envs\rl\Lib\site-packages\gym\spaces\box.py:127: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")
C:\Users\acer\AppData\Local\Temp\ipykernel_20212\4128707195.py:74: UserWarning: Using a target size (torch.Size([64, 64, 1])) that is different to the input size (torch.Size([64, 4])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  q_loss = F.mse_loss(current_q_values, target)


Episode 0, Total Reward: [-976.48557069]
Episode 10, Total Reward: [-881.68396444]
Episode 20, Total Reward: [-881.68305797]
Episode 30, Total Reward: [-882.1604187]
Episode 40, Total Reward: [-881.42135885]
Episode 50, Total Reward: [-881.62037518]
Episode 60, Total Reward: [-885.29735935]
Episode 70, Total Reward: [-879.68812926]
Episode 80, Total Reward: [-882.75932044]
Episode 90, Total Reward: [-810.08701074]
Episode 100, Total Reward: [-882.30522759]
Episode 110, Total Reward: [-885.88757383]
Episode 120, Total Reward: [-881.48372111]
Episode 130, Total Reward: [-884.22254275]
Episode 140, Total Reward: [-884.54115639]
Episode 150, Total Reward: [-876.28498382]
Episode 160, Total Reward: [-884.78417413]
Episode 170, Total Reward: [-883.60456106]
Episode 180, Total Reward: [-883.72060578]
Episode 190, Total Reward: [-884.08508865]
Episode 200, Total Reward: [-882.33489941]
Episode 210, Total Reward: [-881.57966726]
Episode 220, Total Reward: [-885.39618377]
Episode 230, Total Rewa